In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import random

from sklearn.model_selection import train_test_split
from tqdm import tqdm
import cv2
import os
import matplotlib.image as mpimg
def getTrainTest(listSignsDirectory, sampleDiv, pTrain, typeIm):
    
    train = []
    test = []
    trainLabels = []
    testLabels = []
    
    for directory in listSignsDirectory:
        
        relativePath = "../dataset/" + directory
        
        # Get number frames
        nFiles = len(os.listdir(os.path.abspath(relativePath)))   
        
        # Random sample of Data for Train - Test
        sample = random.sample(range(1,nFiles), round(nFiles/sampleDiv))
        nTrain = int(round(round(nFiles/sampleDiv)*pTrain))
        
        print("Reading " + str(round(nFiles/sampleDiv)) + " random frames from " + relativePath)
        
        # Reading random sample
        for frame in sample[:nTrain]:
            img = cv2.imread(relativePath + "/" + directory + "_" + str(frame) + ".jpg", typeIm)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            train.append(img)
            
        for frame in sample[nTrain:]:
            img = cv2.imread(relativePath + "/" + directory + "_" + str(frame) + ".jpg", typeIm)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            test.append(img)
        #Hacemos one_hot encode
        one_hot = [0] * 5
        one_hot[int(directory[-1]) - 1] = 1
        trainLabels.append( [one_hot] * nTrain)
        testLabels.append( [one_hot] * (round(nFiles/sampleDiv) - nTrain))
        
    
    trainLabels = [item for sublist in trainLabels for item in sublist]
    testLabels = [item for sublist in testLabels for item in sublist]
    #Hacemos one_hot encode
    #one_hot = [0] * 5
    #one_hot[code - 1] = 1
    #y.append(one_hot)
    
    print("Shuffling data...\n")   
    zipped = list(zip(train, trainLabels))
    random.shuffle(zipped)
    train, trainLabels = zip(*zipped)

    zipped = list(zip(test, testLabels))
    random.shuffle(zipped)
    test, testLabels = zip(*zipped)
    
    return train, trainLabels, test, testLabels



In [3]:
modelCNN = tf.keras.models.load_model("../models/modelCNN.h5")
listSignsDirectory = ["sign_1", "sign_2", "sign_3", "sign_4", "sign_5"]
train, trainLabels, test, testLabels = getTrainTest(listSignsDirectory, 4, 0.8,cv2.COLOR_BGR2RGB)
train = np.array(train)
trainLabels = np.array(trainLabels)
test = np.array(test)
testLabels = np.array(testLabels)


#Evaluar nuestro modelo ya entrenado, contra el dataset de pruebas
print("Resultado en las pruebas CNN con OHE: ")
test_loss, test_accuracy = modelCNN.evaluate(test, testLabels)




Reading 375 random frames from ../dataset/sign_1
Reading 368 random frames from ../dataset/sign_2
Reading 406 random frames from ../dataset/sign_3
Reading 356 random frames from ../dataset/sign_4
Reading 350 random frames from ../dataset/sign_5
Shuffling data...

Resultado en las pruebas CNN con OHE: 
12/12 [==============================] - 1s 44ms/step - loss: 1.0719 - accuracy: 0.8922


In [10]:
varia = modelCNN.predict(test)
print(varia)
index_max = np.argmax(varia)
print(index_max)


12/12 [==============================] - 0s 39ms/step
[[3.7734602e-10 2.1213687e-05 9.9994880e-01 1.4750735e-05 1.5113605e-05]
 [1.4570297e-15 2.1937586e-14 2.8305839e-12 9.9999994e-01 2.8525652e-13]
 [1.0200785e-07 9.6730453e-08 3.4550899e-07 9.9998957e-01 9.8236405e-06]
 ...
 [1.6689185e-16 9.9999225e-01 2.9902108e-16 2.1749412e-20 7.7998802e-06]
 [5.7117129e-05 9.9994206e-01 7.9424836e-07 2.5585243e-09 2.1855026e-12]
 [9.9982554e-01 1.7302803e-04 1.2071637e-11 1.4847994e-06 8.3328299e-13]]
8
